# Setup

In [1]:
!pip install transformers
!pip install pymongo

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/ef/b5/b6107bd65fa4c96fdf00e4733e2fe5729bb9e5e09997f63074bb43d3ab28/huggingface_hub-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.2 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.15,>=0.14 from https://files.pythonhosted.org/packages/a7/7b/c1f643eb086b6c5c33eef0c3752e37624b

In [2]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for accelerate>=0.19.0 from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/37/1a/8d20e8704da9fa070eb909265584b960da57be1d833d550c59f50906dc5c/peft-0.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 16.9 MB/s eta 0:00:00


In [12]:
!pip3 uninstall auto-gptq -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: auto-gptq 0.1.0
Uninstalling auto-gptq-0.1.0:
  Successfully uninstalled auto-gptq-0.1.0


In [13]:
!pip3 uninstall -y auto-gptq
!git clone https://github.com/PanQiWei/AutoGPTQ
!cd AutoGPTQ
!pip3 install .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3541, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 3541 (delta 536), reused 591 (delta 509), pack-reused 2845
Receiving objects: 100% (3541/3541), 7.75 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (2329/2329), done.
hug

In [17]:
!pip install --upgrade bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
!pip install google-cloud-storage

In [3]:
!mkdir /home/jupyter/qa_dataset

In [1]:
import pymongo

def download_index(index_name):
    # Define MongoDB connection settings
    username = "google_colab"
    password = "TEBvOpN9x3n5TZhE"
    cluster_url = "masterthesis.2tj9bf6.mongodb.net"

    # Construct the MongoDB connection string
    connection_string = f"mongodb+srv://{username}:{password}@{cluster_url}/test?retryWrites=true&w=majority"

    client = pymongo.MongoClient(connection_string)
    try:
        # Create a MongoDB client

        # Access the specified database
        database_name = index_name  # Assume the database name is the same as the index name
        db = client[database_name]

        # Retrieve all documents from the database
        document_entries = {}

        for document in db["documents"].find({}):
            document_name = document["documentName"]
            document_id = document["documentID"]

            if (document_name, document_id) not in document_entries:
                document_entries[(document_name, document_id)] = {
                    "documentName": document_name,
                    "documentID": document_id,
                    "passages": []
                }

            # Apply the pipeline to the "context" and append to passages
            document_entries[(document_name, document_id)]["passages"].append(document["context"])

        # Convert the dictionary values to a list of document entries
        documents = list(document_entries.values())

        # Construct the index object
        index = {
            "database": database_name,
            "description": db["metadata"].find_one()["description"],
            "documents": documents
        }

        return index

    except pymongo.errors.ConnectionFailure as e:
        print(f"MongoDB Connection Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'client' in locals():
            client.close()


In [57]:
# Example usage:
#index_german_test = download_index("IndexGermanTest")
index_english_test = download_index("IndexEnglishTest")
#index_german = download_index("IndexGerman")
index_english = download_index("IndexEnglish")
#index_english_translation = download_index("IndexEnglishTranslation")

In [3]:
def append_document_name_to_passages(index, translation_func=None):
    new_index = index.copy()

    for document in new_index["documents"]:
        document_name = document["documentName"]

        if translation_func:
            translated_name = translation_func(document_name)
        else:
            translated_name = document_name
        
        new_passages = []
        for passage in document["passages"]:
            new_passages.append(passage + " - " + translated_name)
            #passage["content"] = passage["content"] + " " + translated_name
        document["passages"] = new_passages

    return new_index

In [4]:
index_english_test = append_document_name_to_passages(index_english_test)

In [5]:
index_english_test

{'database': 'IndexEnglishTest',
 'description': 'Test Index English only DnCS - chunck size 240 snippets + paragraphs \n\n  + filter low- visual extract - german small set',
 'documents': [{'documentName': 'Master Data and Computer Science',
   'documentID': '419b0cb7-0324-4adc-b7a3-c16919a0d68d',
   'passages': ["Study and examination regulations of Heidelberg University for the Master's degree in Data and Computer Science - Master Data and Computer Science",
    'dated July 22, 2010 in the version of September 29, 2021 - Master Data and Computer Science',
    'Based on Section 32 of the Law on Universities in Baden-Wirttemberg (State Higher Education Act - LHG), last amended by Article 1 of the Fourth Law amending higher education regulations (Fourth Higher Education Law Amendment Act - 4th HRAG) of December 17, 2020 (GBI . - Master Data and Computer Science',
    'p. 1204, 1230), the Senate of Heidelberg University passed the following statutes on September 28, 2021. - Master Data 

In [48]:
# Crop to just have 10 passages
if 'documents' in index_english_test and index_english_test['documents']:
    document = index_english_test['documents'][0]
    if 'passages' in document:
        document['passages'] = document['passages'][:10]

# Resulting index object with only 10 passages
print(index_english_test)

{'database': 'IndexEnglishTest', 'description': 'Test Index English only DnCS - chunck size 240 snippets + paragraphs \n\n  + filter low- visual extract - german small set', 'documents': [{'documentName': 'Master Data and Computer Science', 'documentID': '419b0cb7-0324-4adc-b7a3-c16919a0d68d', 'passages': ["Study and examination regulations of Heidelberg University for the Master's degree in Data and Computer Science - Master Data and Computer Science", 'dated July 22, 2010 in the version of September 29, 2021 - Master Data and Computer Science', 'Based on Section 32 of the Law on Universities in Baden-Wirttemberg (State Higher Education Act - LHG), last amended by Article 1 of the Fourth Law amending higher education regulations (Fourth Higher Education Law Amendment Act - 4th HRAG) of December 17, 2020 (GBI . - Master Data and Computer Science', 'p. 1204, 1230), the Senate of Heidelberg University passed the following statutes on September 28, 2021. - Master Data and Computer Science

# QA-Generation

In [12]:
import json
import time
import torch

def generate_questions_for_index(index, examples, pipeline, save_path, num_questions):
    # Extract the passages from the index
    documents = index.get("documents", [])
    contexts = []
    for document in documents:
        contexts += document.get("passages", [])

    generated_question_tuples = []

    # Collect all examples
    example_contexts, example_questions, example_answers = zip(*examples)

    # Create a list of prompts
    prompts = []
    for i, (example_context, example_question, example_answer) in enumerate(examples):
        prompt = f'''
[INST] <<SYS>>
You are an assistant who generates one question given a context. Please start your generated question with the indicator 'Generated Question:' Here are some examples:

Example {i + 1}:
Context: {example_context}
Generated Question: {example_question}

'''
    for context in contexts:
        prompt += f"<<SYS>>\nGenerate the question based on the following context:\n\nContext: {context}[/INST]"
        prompts.append(prompt)
    
    total_contexts = len(contexts)

    start_time = time.time()

    # Process data through the pipeline
    generated_characters = 0

    for i, prompt in enumerate(prompts):
        question = []
        for _ in range(num_questions):
            generated_text = pipeline(prompt, num_return_sequences=1)[0]
            generated_characters += len(generated_text["generated_text"])
            if len(generated_text["generated_text"]) < 400:
                question.append(generated_text["generated_text"].split("Generated Question:")[-1].strip())


            
        generated_question_tuples.append((contexts[i], question))

        elapsed_time = time.time() - start_time

        # Calculate progress percentage (multiply by 2 to reach 100%)
        progress = (i + 1) / (total_contexts) * 100

        # Print progress and elapsed time
        print(f"Progress: {progress:.2f}%, Elapsed Time: {elapsed_time:.2f} seconds, Generated Characters: {generated_characters}")
        
        torch.cuda.empty_cache()
        
    total_time = time.time() - start_time

    # Organize the data into a list of dictionaries
    dataset = [{"context": passage, "question": question} for (passage, question) in generated_question_tuples]

    # Save the dataset to a JSON file
    file_name = index.get("database", "index") + ".json"
    file_path = f"{save_path}/{file_name}"
    with open(file_path, "w", encoding="utf-8") as json_file:
        json.dump(dataset, json_file, ensure_ascii=False, indent=4)

    print(f"Question-Context dataset saved to: {file_path}")
    print(f"Total Time: {total_time:.2f} seconds")
    return file_path

In [47]:
import json
import time
import torch

def generate_questions_for_index(index, examples, pipeline, save_path, num_questions):
    # Extract the passages from the index
    documents = index.get("documents", [])
    contexts = []
    for document in documents:
        contexts += document.get("passages", [])

    generated_question_tuples = []

    # Collect all examples
    example_contexts, example_questions, example_answers = zip(*examples)

    # Create a list of prompts
    prompts = []
    for i, (example_context, example_question, example_answer) in enumerate(examples):
        prompt = f'''
[INST] <<SYS>>
You are an assistant who generates one question given a context. Please start your generated question with the indicator 'Generated Question:' Here are some examples:

Example {i + 1}:
Context: {example_context}
Generated Question: {example_question}

'''
    def data():
        for context in contexts:
            yield prompt + f"<<SYS>>\nGenerate the question based on the following context:\n\nContext: {context}[/INST]"
    
    total_contexts = len(contexts)

    start_time = time.time()

    # Process data through the pipeline
    generated_characters = 0
    
    i = 0
    for generated_text in pipeline(data(), num_return_sequences=1):
        question = []
        generated_characters += len(generated_text[0]["generated_text"])
        question.append(generated_text[0]["generated_text"].split("Generated Question:")[-1].strip())
            
        generated_question_tuples.append((contexts[i], question))

        elapsed_time = time.time() - start_time

        # Calculate progress percentage (multiply by 2 to reach 100%)
        progress = (i + 1) / (total_contexts) * 100

        # Print progress and elapsed time
        print(f"Progress: {progress:.2f}%, Elapsed Time: {elapsed_time:.2f} seconds, Generated Characters: {generated_characters}")
        
#        torch.cuda.empty_cache()
        i += 1
        
    total_time = time.time() - start_time

    # Organize the data into a list of dictionaries
    dataset = [{"context": passage, "question": question} for (passage, question) in generated_question_tuples]

    # Save the dataset to a JSON file
    file_name = index.get("database", "index") + ".json"
    file_path = f"{save_path}/{file_name}"
    with open(file_path, "w", encoding="utf-8") as json_file:
        json.dump(dataset, json_file, ensure_ascii=False, indent=4)

    print(f"Question-Context dataset saved to: {file_path}")
    print(f"Total Time: {total_time:.2f} seconds")
    return file_path

## Question Generation (seq-2-seq) - English

In [10]:
# Example function to generate a question from context
def generate_question_from_context_s2s(context):
    # Initialize the pipeline for question generation
    question_generation_pipeline = pipeline("text2text-generation", model="voidful/context-only-question-generator")

    generated_question = question_generation_pipeline(context)

    return generated_question[0]["generated_text"], None

In [12]:
colab_storage_path = '/home/jupyter/qa_dataset'
file_path = generate_questions_for_index(index_english_test, generate_question_from_context_s2s, colab_storage_path)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question-Context-Answer dataset saved to: /home/jupyter/qa_dataset/IndexEnglishTest.json


## Question Answer Generation (Llama2-7B) - English

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [7]:
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

use_triton = False
model_file = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_file, use_fast=True)
print("Loaded Tokenizer")

model = AutoGPTQForCausalLM.from_quantized(model_file,
    use_safetensors=True,
    trust_remote_code=True,
    device_map="auto",
    use_triton=use_triton
)

print("Initialized model")

Loaded Tokenizer


Initialized model


In [14]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    temperature=0.8,
    top_p=0.95,
    top_k=20,
    device_map="auto",
    repetition_penalty=1.1,
    max_new_tokens=546,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

In [38]:
torch.cuda.empty_cache()

In [53]:
#from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
# To use a different branch, change revision
#model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                             device_map="auto",
#                                             trust_remote_code=False,
#                                             revision="main")

#tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''

#print("\n\n*** Generate:")

#input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
#output = model.generate(inputs=input_ids, do_sample=True, temperature=0.7, top_p=0.95, max_new_tokens=512)
#print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    num_return_sequences = 3,
#    temperature=0.9,
#    top_p=0.95,
#    top_k=20,
#    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

*** Pipeline:


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=500,
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
def generate_question_from_context_llama(context, examples, pipeline):
    # Create a dynamic prompt based on the provided examples
    prompt_template = "Generate a question given a context. Here are some examples:\n\n"
    for i, (example_context, example_question, example_answer) in enumerate(examples):
        prompt_template += f"Example {i + 1}:\nContext: {example_context}\nGenerated Question: {example_question}\n\n"
    prompt_template += f"Generate the question based on the following context:\n\nContext: {context}"

    #prompt_template = f"Generate a question given the following context: {context}"
    # Generate a question based on the dynamic prompt
    generated_text = pipeline(prompt_template, num_return_sequences=1, max_length=500)

    #print(f"Text: {generated_text}")
    
    # Extract and return the generated question and answer
    generated_text = generated_text[0]["generated_text"]
    question = generated_text.split("\nGenerated Question:")[-1].strip()
    #answer = generated_text[-2].replace("Generated Answer:", "").strip()

    #return question, answer
    return question

In [23]:
context = "(1) The consecutive, research-oriented master's program in Data and Computer Science provides in-depth expertise and scientific methods in computer science and, at the discretion of the students, in related fields."


generate_question_from_context_llama(context, pipeline)

Text: [{'generated_text': "Generate a question given a context. Here are some examples:\n\nExample 1:\nContext: The master's program in Data and Computer Science includes an application area. Annex 3 lists the possible application areas. Upon request, the examination board can also approve a different application area.\nGenerated Question: Can I choose application areas that are not listed?\n\nExample 2:\nContext: (5) The final failure in a mandatory module leads to the loss of the examination claim. In elective mandatory modules, if provided for in the module handbook, the failure can be compensated for by the successful completion of another elective mandatory module or another performance within the respective module. § 4 Paragraph 2 remains unaffected.\nGenerated Question: When does the failure of an exam lead to the termination of the study?\n\nExample 3:\nContext: (3) The application must be made in writing to the examination board. It is the responsibility of the applicant to pr

"Can I choose a related field for my master's program?"

In [9]:
# Example list of examples
example1 = ("The master's program in Data and Computer Science includes an application area. Annex 3 lists the possible application areas. Upon request, the examination board can also approve a different application area. - Master Data and Computer Science",
            "I study Data and Computerscience. Can I choose application areas that are not listed?",
            "Yes, upon request, the examination board can also approve a different application area.")

example2 = ("(5) The final failure in a mandatory module leads to the loss of the examination claim. In elective mandatory modules, if provided for in the module handbook, the failure can be compensated for by the successful completion of another elective mandatory module or another performance within the respective module. § 4 Paragraph 2 remains unaffected. - Bachelor Physics",
            "When does the failure of an exam lead to the termination of the study?",
            "For Bachelor Students of Phisics, the final failure in a mandatory module leads to the loss of the examination claim. In elective mandatory modules, this can be compensated for.")

example3 = ("(3) The application must be made in writing to the examination board. It is the responsibility of the applicant to provide the necessary information about the performance to be recognized. The burden of proof for the existence of a significant difference in academic achievements lies with Heidelberg University; the obligations of the applicant, especially according to Sentence 1 and Sentence 2, remain unaffected. The burden of proof for the existence of equivalence in non-academic achievements lies with the applicant. - Bachelor Philosophy",
            "How can I obtain recognition of study achievements in Heidelberg?",
            "Recognition of study achievements requires a written application to the examination board. The applicant is responsible for providing all necessary information regarding the performance to be recognized.")

example4 = ("The intermediate examination consists of successful participation in the exercises for beginners in the subjects Civil Law, Public Law, and Criminal Law. The partial performances of the exercise (homework and supervisory work under examination conditions) must in principle be performed in the exercise of a semester; § 4 paragraph 5 remains unaffected.",
            "What are the components of the intermediate examination in Civil Law, Public Law, and Criminal Law?",
            "The intermediate examination in Civil Law, Public Law, and Criminal Law consists of successful participation in exercises for beginners, which includes completing homework and supervisory work under examination conditions during the semester.")


examples = [example1, example2, example3, example4]

context = "(1) The consecutive, research-oriented master's program in Data and Computer Science provides in-depth expertise and scientific methods in computer science and, at the discretion of the students, in related fields."

#question = generate_question_from_context_llama(context, examples, pipeline)

#print("Generated Question:", question)

In [31]:
torch.cuda.empty_cache()

In [49]:
qa_dataset_path = generate_questions_for_index(index_english_test, examples, pipe, "/home/jupyter/qa_dataset", 3)

Progress: 10.00%, Elapsed Time: 1.12 seconds, Generated Characters: 1048
Progress: 20.00%, Elapsed Time: 2.37 seconds, Generated Characters: 2017
Progress: 30.00%, Elapsed Time: 4.03 seconds, Generated Characters: 3254
Progress: 40.00%, Elapsed Time: 5.78 seconds, Generated Characters: 4339
Progress: 50.00%, Elapsed Time: 6.77 seconds, Generated Characters: 5281
Progress: 60.00%, Elapsed Time: 7.99 seconds, Generated Characters: 6311
Progress: 70.00%, Elapsed Time: 9.40 seconds, Generated Characters: 7492
Progress: 80.00%, Elapsed Time: 10.47 seconds, Generated Characters: 8662
Progress: 90.00%, Elapsed Time: 11.80 seconds, Generated Characters: 9715
Progress: 100.00%, Elapsed Time: 13.40 seconds, Generated Characters: 10879
Question-Context dataset saved to: /home/jupyter/qa_dataset/IndexEnglishTest.json
Total Time: 13.40 seconds


In [43]:
from google.cloud import storage

In [44]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [54]:
bucket_name = "masterthesis_gertz"
source_file_path = qa_dataset_path
destination_path = "qa_dataset/" + source_file_path.split("/")[-1]

upload_blob(bucket_name, source_file_path, destination_path)

File /home/jupyter/qa_dataset/IndexEnglishTest.json uploaded to qa_dataset/IndexEnglishTest.json.


In [ ]:
bucket_name = "masterthesis_gertz"
base_destination_path = "qa_dataset/"

for i in range(1, 4):  # Run the loop 3 times
    index = index_english_test
    index["database"] = f"IndexEnglishTest{i}"
    qa_dataset_path = generate_questions_for_index(index, examples, pipe, base_destination_path, i)
    source_file_path = qa_dataset_path
    destination_path = f"qa_dataset/{source_file_path.split('/')[-1]}"
    upload_blob(bucket_name, source_file_path, destination_path)

Progress: 0.26%, Elapsed Time: 1.47 seconds, Generated Characters: 1056
Progress: 0.52%, Elapsed Time: 3.09 seconds, Generated Characters: 1993
Progress: 0.78%, Elapsed Time: 5.29 seconds, Generated Characters: 3285
Progress: 1.04%, Elapsed Time: 7.03 seconds, Generated Characters: 4322
Progress: 1.30%, Elapsed Time: 7.96 seconds, Generated Characters: 5223
Progress: 1.55%, Elapsed Time: 9.01 seconds, Generated Characters: 6185
Progress: 1.81%, Elapsed Time: 10.87 seconds, Generated Characters: 7408
Progress: 2.07%, Elapsed Time: 12.40 seconds, Generated Characters: 8604
Progress: 2.33%, Elapsed Time: 13.30 seconds, Generated Characters: 9549
Progress: 2.59%, Elapsed Time: 15.05 seconds, Generated Characters: 10724
Progress: 2.85%, Elapsed Time: 16.78 seconds, Generated Characters: 11757
Progress: 3.11%, Elapsed Time: 18.01 seconds, Generated Characters: 12772
Progress: 3.37%, Elapsed Time: 19.60 seconds, Generated Characters: 13937
Progress: 3.63%, Elapsed Time: 21.65 seconds, Generat

## Question Answer Generation (Igel) - Deutsch

In [15]:
from transformers import pipeline
import torch
pipe = pipeline("text-generation", model="philschmid/instruct-igel-001", tokenizer="philschmid/instruct-igel-001", device_map="auto", max_new_tokens=400)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 from transformers import pipeline                                                            │
│   2 import torch                                                                                 │
│ ❱ 3 pipe = pipeline("text-generation", model="philschmid/instruct-igel-001", tokenizer="phil     │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/pipelines/__init__.py:795 in pipeline        │
│                                                                                                  │
│   792 │   │   framework=framework,                                                               │
│   793 │   │   task=task,                                                                         │
│   794 │   │   **hub_kwargs,                                                                      │
│ ❱ 795 │   │   **model_kwargs,                                                                    │
│   796 │   )                                                                                      │
│   797 │                                                                                          │
│   798 │   model_config = model.config                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:278 in                     │
│ infer_framework_load_model                                                                       │
│                                                                                                  │
│    275 │   │   │   │   continue                                                                  │
│    276 │   │                                                                                     │
│    277 │   │   if isinstance(model, str):                                                        │
│ ❱  278 │   │   │   raise ValueError(f"Could not load model {model} with any of the following cl  │
│    279 │                                                                                         │
│    280 │   framework = infer_framework(model.__class__)                                          │
│    281 │   return framework, model                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Could not load model philschmid/instruct-igel-001 with any of the following classes: (<class 
'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 
'transformers.models.bloom.modeling_bloom.BloomForCausalLM'>).

In [13]:
def generate_question_from_context_igel(context, examples, pipe):
    # Initialize the pipeline for question generation using "igel" model

    # Create a dynamic German prompt based on the provided examples
    prompt_template = "Generiere eine Frage und eine Antwort auf Basis des gegebenen Kontexts. Hier sind einige Beispiele:\n\n"
    for i, (example_context, example_question, example_answer) in enumerate(examples):
        prompt_template += f"Beispiel {i + 1}:\nKontext: {example_context}\nGenerierte Frage: {example_question}\nGenerierte Antwort: {example_answer}\n\n"
    prompt_template += f"Deine Aufgabe ist eine Frage und Antwort basierend auf folgendem Kontext zu generieren:\n\nKontext: {context}\n Generierte Frage: __ \nGenerierte Antwort: __"

    # Generate a question based on the dynamic German prompt
    generated_text = pipe(prompt_template)  # You can adjust the max_length as needed
    
    print(f"text: {generated_text}")
    # Extract and return the generated question and answer
    generated_text = generated_text[0]["generated_text"].split("\n")
    question = generated_text[-4].replace("Generierte Frage:", "").strip()
    answer = generated_text[-2].replace("Generierte Antwort:", "").strip()

    return question, answer

In [14]:
# Example list of examples
example1 = ("Das Masterstudium der Data and Computer Science beinhaltet ein Anwendungsgebiet. Anlage 3 listet die möglichen Anwendungsgebiete auf. Auf Antrag kann der Prüfungsausschuss statt diesen auch ein anderes Anwendungsgebiet genehmigen.", "Kann ich Anwendungsgebiete belegen, die nicht aufgelistet sind?", "Ja, auf Antrag kann der Prüfungsausschuss auch ein anderes Anwendungsgebiet genehmigen.")
example2 = ("(5) Das endgültige Nichtbestehen eines Pflichtmoduls führt zum Verlust des Prüfungsan- spruchs. Bei Wahlpflichtmodulen kann, soweit dies im Modulhandbuch vorgesehen ist, das Nichtbestehen durch die erfolgreiche Absolvierung eines anderen Wahlpflichtmoduls oder einer anderen Leistung innerhalb des betreffenden Moduls ausgeglichen werden. § 4 Absatz 2 bleibt unberührt.", "Wann führt eine nicht bestandene Prüfung zum Abbruch des Studiums?", "Das endgültige Nichtbestehen eines Pflichtmoduls führt zum Verlust des Prüfungsanspruchs. Bei Wahlpflichtmodulen kann dies ausgeglichen werden.")
example3 = (
    "(3) Der Antrag ist schriftlich beim Prüfungsausschuss zu stellen. Es obliegt der antragstellenden Person, die erforderlichen Informationen über die anzuerkennende Leistung bereitzustellen. Die Darlegungs- und Beweislast für das Vorliegen eines wesentlichen Unterschieds bei hochschulischen Leistungen liegt bei der Ruprecht-Karls-Universität Heidelberg; Mitwirkungspflichten der antragstellenden Person, insbesondere nach Satz 1 und Satz 2, bleiben hiervon unberührt. Die Darlegungs- und Beweislast für das Vorliegen von Gleichwertigkeit bei außerhochschulischen Leistungen liegen bei der antragstellenden Person.",
    "Wie erhalte ich die Anerkennung von Studienleistungen in Heidelberg?",
    "Die Anerkennung von Studienleistungen erfordert einen schriftlichen Antrag beim Prüfungsausschuss. Die antragstellende Person ist dafür verantwortlich, alle erforderlichen Informationen bezüglich der anzuerkennenden Leistung bereitzustellen."
)
examples = [example1, example2, example3]

context = "(1) Der konsekutive, forschungsorientierte Masterstudiengang Data and Computer Sci- ence vermittelt tiefergehendes Fachwissen und wissenschaftliche Methoden der Infor- matik und nach Wahl der Studierenden auch angrenzender F achgebiete."

question, answer = generate_question_from_context_igel(context, examples, pipe)
print("Generierte Frage:", question)
print("Generierte Antwort:", answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Input length of input_ids is 336, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


text: [{'generated_text': 'Generiere eine Frage und eine Antwort auf Basis des gegebenen Kontexts. Hier sind einige Beispiele:\n\nBeispiel 1:\nKontext: Das Masterstudium der Data and Computer Science beinhaltet ein Anwendungsgebiet. Anlage 3 listet die möglichen Anwendungsgebiete auf. Auf Antrag kann der Prüfungsausschuss statt diesen auch ein anderes Anwendungsgebiet genehmigen.\nGenerierte Frage: Kann ich Anwendungsgebiete belegen, die nicht aufgelistet sind?\nGenerierte Antwort: Ja, auf Antrag kann der Prüfungsausschuss auch ein anderes Anwendungsgebiet genehmigen.\n\nBeispiel 2:\nKontext: (5) Das endgültige Nichtbestehen eines Pflichtmoduls führt zum Verlust des Prüfungsan- spruchs. Bei Wahlpflichtmodulen kann, soweit dies im Modulhandbuch vorgesehen ist, das Nichtbestehen durch die erfolgreiche Absolvierung eines anderen Wahlpflichtmoduls oder einer anderen Leistung innerhalb des betreffenden Moduls ausgeglichen werden. § 4 Absatz 2 bleibt unberührt.\nGenerierte Frage: Wann führt 

In [26]:
index_german_test

{'database': 'IndexGermanTest',
 'description': 'Test Index only DnCS - chunck size 240 snippets + paragraphs \n\n  + filter low- visual extract - german small set',
 'documents': [{'documentName': 'Studien- und Prüfungsordnung Data Computer Science MSc (2022-10-05)',
   'documentID': 'efb7b236-4379-4a16-a331-a78a62387982',
   'passages': ['Studien- und Prifungsordnung der Ruprecht-Karls-Universitat Heidelberg fur den Masterstudiengang Data and Computer Science',
    'vom 22.\n\nJ uli 2010, geandert am 7.\n\nFebruar 2013, am 29.\n\nSeptember 2021 und zuletzt am 5.\n\nOktober 2022',
    'Aufgrund von §§ 32 Abs.\n\n3 Satz 1, 19 Abs.\n\n1 Nr.\n\n7 des Landeshochschulgesetzes in der Fas- sung vom 1.\n\nJ anuar 2005 (GBI, S. 1), zuletzt geandert am 21.\n\nDezember 2021 (GBI.\n\n2022 S. 1) hat der Senat der Universitat Heidelberg am 4.',
    'Oktober 2022 die nachfolgende Satzung beschlossen.',
    'Der Rektor hat seine Zustimmung am 5.\n\nOktober 2022 erteilt.',
    'Inhalt Abschnitt I: All